In [1]:
#!/usr/bin/python
import cv2, os
import numpy as np
from PIL import Image
import csv
import pandas as pd
import pdb as pdb
from sklearn.cross_validation import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn import svm
import time

settings = {
    'minNeighbors': 2, 
    'minSize': (40,40)
}
image_list = pd.read_csv('driver_imgs_list.csv')
train_subject_subset = image_list.subject.value_counts()[-9:].index.values
test_subject_subset = image_list.subject.value_counts()[:-9].index.values
train = image_list[image_list.subject.isin(train_subject_subset)].reset_index(drop=True)
test = image_list[image_list.subject.isin(test_subject_subset)].reset_index(drop=True)

#take 25% of the sets 
mask = np.random.choice([False, True], len(train), p=[0.75, 0.25])
train = train[mask]
mask = np.random.choice([False, True], len(test), p=[0.75, 0.25])
test = test[mask]
print 'done getting train/test sets'
print 'getting images from paths'

done getting train/test sets
getting images from paths


/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
images = []
predict_images = []
labels = [] 
predict_labels = []

t0 = time.time()
scaled_size = (200,150)

for i, image in train.iterrows():
    image_path = './train/' + image.classname + '/' + image.img
    predict_image_pil = Image.open(image_path).convert('L') #greyscale
    predict_image_pil.thumbnail(scaled_size, Image.ANTIALIAS) #resize
    image = np.array(predict_image_pil, 'uint8') #to array
    images.append(image.flatten())
    labels.append(train['classname'][i])
    
for i, image in test.iterrows():
    image_path = './train/' + image.classname + '/' + image.img
    predict_image_pil = Image.open(image_path).convert('L') #greyscale
    predict_image_pil.thumbnail(scaled_size, Image.ANTIALIAS) #resize
    predict_image = np.array(predict_image_pil, 'uint8') #to array
    predict_images.append(predict_image.flatten())
    predict_labels.append(test['classname'][i])
print('done')




done


In [3]:
# Extract relevant data
training_data = images
training_labels = labels
prediction_data = predict_images
prediction_labels = predict_labels

training_labels = map(lambda each:int(each.strip("c")), training_labels)
prediction_labels = map(lambda each:int(each.strip("c")), prediction_labels)

training_data = map(lambda each:each.flatten(), training_data)
prediction_data = map(lambda each:each.flatten(), prediction_data)

print("done in {}s".format(time.time() - t0))
print ("training classifier")

done in 79.1239788532s
training classifier


In [4]:
from sklearn.multiclass import OneVsOneClassifier
t0 = time.time()
clf = svm.SVC()
clf = clf.fit(training_data, training_labels)
print("done in {}s".format(time.time() - t0))
print("making predictions")

done in 154.87045598s
making predictions


In [5]:
t0 = time.time()
correct = 0
incorrect = 0
pred = clf.predict(prediction_data)
correct = np.sum(pred == prediction_labels)
print 'accuracy using OneVsOneClassifier: {} / {} %'.format(correct, len(prediction_labels))
print("done in {}s".format(time.time() - t0))
print ("making predictions")
    

accuracy using OneVsOneClassifier: 414 / 4044 %
done in 241.784584045s
making predictions


In [ ]:
t0 = time.time()
correct = 0
pred = OneVsOneClassifier(clf.predict(prediction_data))

correct = np.sum(pred == prediction_labels)

print 'accuracy using SVM one vs one: {} out of %'.format(correct, len(prediction_labels))
print("done in {}s".format(time.time() - t0))
print ("making predictions")
    

accuracy using SVM one vs one: 0 out of %
done in 243.964724779s
making predictions


In [ ]:
from sklearn.multiclass import OneVsRestClassifier
t0 = time.time()
correct = 0
pred = OneVsRestClassifier(clf.predict(prediction_data))
pdb.set_trace()
correct = np.sum(pred == prediction_labels)
print 'accuracy using SVM one vs rest: {}%'.format((100*correct)/(correct + incorrect))
print("done in {}s".format(time.time() - t0))
    

--Return--
> <ipython-input-7-d32c602c5309>(5)<module>()->None
-> pdb.set_trace()
